In [1]:
from bson.son import SON
import json
from pymongo import MongoClient
from bson.objectid import ObjectId
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime

## Connect to remote mongodb

In [2]:
from sshtunnel import SSHTunnelForwarder
import pymongo

In [3]:
with open('../keys.json') as k:
    keys = json.loads(k.read())
    MONGO_HOST = keys["SSH"]["MONGO_HOST"] # i.e. 10.9.13.14
    MONGO_DB = keys["SSH"]["MONGO_DB"] # i.e. dm_project 
    MONGO_USER = keys["SSH"]["MONGO_USER"] # i.e. studente
    MONGO_PASS = keys["SSH"]["MONGO_PASS"] # i.e. la password della vm

In [4]:
server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_username=MONGO_USER,
    ssh_password=MONGO_PASS,
    remote_bind_address=('127.0.0.1', 27017)
)

In [5]:
server.start() # remember to stop

In [6]:
client = pymongo.MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port

In [7]:
db = client[MONGO_DB]

In [8]:
db.list_collection_names()

['system.indexes', 'twitch', 'twitter']

# Integration of previous locally dowloaded data. 
### Script on VM started on 23/06/2019

In [9]:
# with open('20190613_0932_data.json', "r", encoding = 'utf8') as json_file:
#     inserted_ids = []
#     for line in json_file:
#         data = json.loads(line)
#         post_id = db.twitch.insert_one(data).inserted_id
#         inserted_ids.append(post_id)

# Count of documents in each collection

## Twitch

In [10]:
db.twitch.count()

5857

## Twitter

In [11]:
db.twitter.count()

68900

# Querying Twitch

In [12]:
pipeline = [{"$project":
             {'_id': 0,
              'time':'$timestamp'} }]

In [13]:
tsmps = list(db.twitch.aggregate(pipeline))

In [14]:
len(tsmps)

5857

In [15]:
start = tsmps[3]['time'] # this is the oldest timestamp
end = tsmps[-1]['time'] # this is the most recent one
start = datetime.strptime(start, "%Y-%m-%d %H:%M:%S.%f")
end = datetime.strptime(end, "%Y-%m-%d %H:%M:%S.%f")

start, end

(datetime.datetime(2019, 6, 13, 9, 32, 34, 348514),
 datetime.datetime(2019, 6, 25, 14, 31, 34, 77487))

In [16]:
(end-start).total_seconds() / 180

5859.665160961112

The following is the list of games we are interested in, so that we can restrict the queries on them if we want.

In [17]:
"""top_games = ['Apex Legends', 'Auto Chess', 'Bio Inc. Redemption', 'Call of Duty: Advanced Warfare',
             'Call of Duty: Black Ops', 'Call of Duty: Black Ops 4',
             'Call of Duty: Black Ops II', 'Call of Duty: Black Ops III',
             'Call of Duty: Ghosts', 'Call of Duty: Modern Warfare 3',
             'Clam Man', 'Counter-Strike: Global Offensive',
             'Dead by Daylight', 'Death Stranding', 'Dota 2', 'Duck Hunt', 'FIFA 19', 'Fortnite',
             'Grand Theft Auto IV', 'Grand Theft Auto V', 'Grand Theft Auto: San Andreas', 'Hearthstone',
             'Just Chatting', 'Layers of Fear 2', 'League of Legends', 'Magic: The Gathering',
             'Mario Kart DS', 'Mario Kart Wii', 'Minecraft', 'Modern Warfare 2',
             'Music & Performing Arts', 'New Super Mario Bros.',
             'New Super Mario Bros. Wii', 'Overwatch',
             "PLAYERUNKNOWN'S BATTLEGROUNDS", 'Pokémon Gold/Silver',
             'Pokémon Red/Blue', 'Slots', 'Splitgate: Arena Warfare', 'Super Mario Bros.',
             'Super Mario World', 'Talk Shows & Podcasts', 'Tetris', 'The Elder Scrolls V: Skyrim',
             "Tom Clancy's Rainbow Six: Siege", 'Total War: Three Kingdoms',
             'Wii Play', 'Wii Sports', 'Wii Sports Resort', 'World of Warcraft']"""

'top_games = [\'Apex Legends\', \'Auto Chess\', \'Bio Inc. Redemption\', \'Call of Duty: Advanced Warfare\',\n             \'Call of Duty: Black Ops\', \'Call of Duty: Black Ops 4\',\n             \'Call of Duty: Black Ops II\', \'Call of Duty: Black Ops III\',\n             \'Call of Duty: Ghosts\', \'Call of Duty: Modern Warfare 3\',\n             \'Clam Man\', \'Counter-Strike: Global Offensive\',\n             \'Dead by Daylight\', \'Death Stranding\', \'Dota 2\', \'Duck Hunt\', \'FIFA 19\', \'Fortnite\',\n             \'Grand Theft Auto IV\', \'Grand Theft Auto V\', \'Grand Theft Auto: San Andreas\', \'Hearthstone\',\n             \'Just Chatting\', \'Layers of Fear 2\', \'League of Legends\', \'Magic: The Gathering\',\n             \'Mario Kart DS\', \'Mario Kart Wii\', \'Minecraft\', \'Modern Warfare 2\',\n             \'Music & Performing Arts\', \'New Super Mario Bros.\',\n             \'New Super Mario Bros. Wii\', \'Overwatch\',\n             "PLAYERUNKNOWN\'S BATTLEGROUNDS"

Alternative (and I think more elegant) way to do it:

In [18]:
with open("../top_50_games_to_monitor_n.json", "r") as f:
    game_list = json.load(f)

    top_games = game_list["selected_top_50_games"]["name"]
top_games

['Fortnite',
 'Call of Duty: Black Ops II',
 'Grand Theft Auto: San Andreas',
 'Clam Man',
 'Wii Play',
 'Modern Warfare 2',
 'Super Mario Bros.',
 'Apex Legends',
 'Duck Hunt',
 'Dead by Daylight',
 'Super Mario World',
 'Call of Duty: Modern Warfare 3',
 'Call of Duty: Ghosts',
 'Grand Theft Auto V',
 'Call of Duty: Black Ops',
 'Death Stranding',
 'Bio Inc. Redemption',
 'Hearthstone',
 'Slots',
 'Just Chatting',
 'Mario Kart Wii',
 'Total War: Three Kingdoms',
 'Mario Kart DS',
 'League of Legends',
 'Magic: The Gathering',
 'Overwatch',
 'FIFA 19',
 'Wii Sports',
 'Call of Duty: Black Ops 4',
 'Pokémon Gold/Silver',
 'World of Warcraft',
 "Tom Clancy's Rainbow Six: Siege",
 'Counter-Strike: Global Offensive',
 'The Elder Scrolls V: Skyrim',
 'Minecraft',
 "PLAYERUNKNOWN'S BATTLEGROUNDS",
 'Dota 2',
 'Call of Duty: Black Ops III',
 'Layers of Fear 2',
 'Tetris',
 'Call of Duty: Advanced Warfare',
 'Splitgate: Arena Warfare',
 'Wii Sports Resort',
 'Talk Shows & Podcasts',
 'Pokémon

Average daily views for each game:

In [19]:
pipeline = [{"$unwind": "$data"},
            {"$match" : { "data.game_name" : { "$in" : top_games },  }},
            { "$group": {
                "_id": {"name": "$data.game_name",
                        "year" :  { "$substr" : ["$timestamp", 0, 4 ] }, 
                        "month" : { "$substr" : ["$timestamp", 5, 2 ] },     
                        "day" :   { "$substr" : ["$timestamp", 8, 2 ] } },
                "avgViewers": { "$avg": '$data.viewers'}
            }},
            {"$sort": {"_id": 1} }]

In [21]:
# this is pretty slow
from datetime import datetime
start = datetime.now()
result = list(db.twitch.aggregate(pipeline))
end = datetime.now()
end-start

datetime.timedelta(0, 34, 30329)

The following are the games initially chosen as top games, but that have not been collected again on twitch.

In [ ]:
result_set = set([elem['_id']['name'] for elem in result])
top_set = set(top_games)

top_set - result_set

In [ ]:
df = json_normalize(result)
df = df.rename(index=str, columns={"_id.name": "name", "_id.day": "day","_id.month": "month","_id.year": "year"})
df = df[['name', 'day', 'month', 'year', 'avgViewers']]

df.head(20)

In [ ]:
server.stop()

In [ ]:
df[df['name'] == 'Tetris']

We add the column 'daily_top', which is the game's position in the ordered list of daily most streamed games on average.

In [ ]:
df['daily_top']=df.sort_values(['day', 'month', 'year', 'avgViewers'],
                               ascending=[True, True, True, False]).groupby(['day', 'month', 'year']).cumcount()
df['daily_top'] = df['daily_top'] + 1

In [ ]:
df.sort_values(['day', 'month', 'year', 'avgViewers'], ascending=[True, True, True, False]).head(20)

# Twitter

In [24]:
server.start()

In [25]:
client = pymongo.MongoClient('127.0.0.1', server.local_bind_port) # server.local_bind_port is assigned local port

In [26]:
db = client[MONGO_DB]

In [27]:
db.list_collection_names()

['system.indexes', 'twitch', 'twitter']

Daily total retweets and likes for each game:

In [53]:
pipeline = [{ "$group": {
                "_id": {"name": "$query",
                        "day": {"$dayOfMonth": "$date" },
                        "month": { "$month": "$date" },
                        "year": { "$year": "$date" }},
                "tot_retweets": { "$sum": "$retweets" },
                "tot_likes": { "$sum" : "$likes"},
                "tweet_count" : { "$sum": 1 }
            }},
            {"$sort": {"_id": 1} }]

In [54]:
result2 = list(db.twitter.aggregate(pipeline))

df2 = json_normalize(result2)
df2 = df2.rename(index=str, columns={"_id.name": "name", "_id.day": "day","_id.month": "month","_id.year": "year"})
df2 = df2[['name', 'day', 'month', 'year', 'tot_retweets', 'tot_likes', 'tweet_count']]
df2.head(20)

,name,day,month,year,tot_retweets,tot_likes,tweet_count
0,apex legends,13,6,2019,54,349,96
1,apex legends,14,6,2019,140,1022,200
2,apex legends,15,6,2019,160,1154,200
3,apex legends,16,6,2019,173,1491,169
4,apex legends,17,6,2019,112,692,16
5,apex legends,18,6,2019,3517,15684,200
6,apex legends,19,6,2019,493,1366,79
7,apex legends,20,6,2019,615,3137,200
8,apex legends,21,6,2019,421,2873,200
9,apex legends,22,6,2019,228,1229,200


In [55]:
df2[df2['name'] == 'tetris' ]

,name,day,month,year,tot_retweets,tot_likes,tweet_count
526,tetris,13,6,2019,28,282,200
527,tetris,14,6,2019,726,4310,76
528,tetris,15,6,2019,60,597,200
529,tetris,16,6,2019,143,903,200
530,tetris,17,6,2019,175,830,200
531,tetris,18,6,2019,360,1830,110
532,tetris,19,6,2019,317,1547,200
533,tetris,20,6,2019,764,11352,200
534,tetris,21,6,2019,188,1452,200
535,tetris,22,6,2019,38,305,29


We add the column 'daily_top', which is the game's position in the ordered list of daily most liked tweets.

In [56]:
df2['daily_top'] = df2.sort_values(['day', 'month', 'year', 'tot_likes'],
                                   ascending=[True, True, True, False]).groupby(['day', 'month', 'year']).cumcount()
df2['daily_top'] = df2['daily_top'] + 1

In [57]:
df2.sort_values(['day', 'month', 'year', 'tot_likes'], ascending=[True, True, True, False]).head(20)

,name,day,month,year,tot_retweets,tot_likes,tweet_count,daily_top
186,fortnite,13,6,2019,419,7344,200,1
246,just chatting,13,6,2019,170,4403,179,2
310,minecraft,13,6,2019,290,2815,200,3
437,slots,13,6,2019,379,2018,200,4
379,overwatch,13,6,2019,144,696,200,5
12,auto chess,13,6,2019,113,671,56,6
234,hearthstone,13,6,2019,37,588,115,7
262,league of legends,13,6,2019,70,548,200,8
610,world of warcraft,13,6,2019,80,501,161,9
0,apex legends,13,6,2019,54,349,96,10


## Which is the  game with the highest retweet value on a given day?

### Pandas solution

In [64]:
df2.loc[df2.daily_top==1]

,name,day,month,year,tot_retweets,tot_likes,tweet_count,daily_top
186,fortnite,13,6,2019,419,7344,200,1
187,fortnite,14,6,2019,1878,34285,200,1
189,fortnite,16,6,2019,2268,33085,200,1
190,fortnite,17,6,2019,1056,35726,200,1
191,fortnite,18,6,2019,8851,53061,200,1
192,fortnite,19,6,2019,1592,15202,200,1
193,fortnite,20,6,2019,2154,30358,200,1
194,fortnite,21,6,2019,2977,22664,200,1
197,fortnite,24,6,2019,9168,39009,200,1
319,minecraft,22,6,2019,1303,43675,200,1


### MongoDB solution

In [73]:
pipeline_highest_retweet_per_day = [{
		"$group": {
			"_id": {
				"name": "$query",
				"day": {
					"$dayOfMonth": "$date"
				},
				"month": {
					"$month": "$date"
				},
				"year": {
					"$year": "$date"
				}
			},
			"tot_retweets": {
				"$sum": "$retweets"
			}
		}
	},
	{
		"$sort": {
			"tot_retweets": -1
		}
	},
	{
		"$group": {
			"_id": {
				"day": "$_id.day",
				"month": "$_id.month",
				"year": "$_id.year"
			},
			"max_retweets": {
				"$first": "$tot_retweets"
			},
			"game": {
				"$first": "$_id.name"
			}
		}
	},
	{
		"$project": {
			"_id": 0,
			"day": "$_id.day",
			"month": "$_id.month",
			"year": "$_id.year",
			"max_retweets": 1,
			"game": 1

		}
	},
	{
		"$sort": {
			"day": 1
		}
	}
]
highest_retweet_per_day = list(db.twitter.aggregate(pipeline_highest_retweet_per_day))
highest_retweet_per_day

[{'max_retweets': 419,
  'game': 'fortnite',
  'day': 13,
  'month': 6,
  'year': 2019},
 {'max_retweets': 1878,
  'game': 'fortnite',
  'day': 14,
  'month': 6,
  'year': 2019},
 {'max_retweets': 9617,
  'game': 'pokémon gold/silver',
  'day': 15,
  'month': 6,
  'year': 2019},
 {'max_retweets': 3365, 'game': 'slots', 'day': 16, 'month': 6, 'year': 2019},
 {'max_retweets': 6082,
  'game': 'overwatch',
  'day': 17,
  'month': 6,
  'year': 2019},
 {'max_retweets': 8851,
  'game': 'fortnite',
  'day': 18,
  'month': 6,
  'year': 2019},
 {'max_retweets': 1592,
  'game': 'fortnite',
  'day': 19,
  'month': 6,
  'year': 2019},
 {'max_retweets': 4211,
  'game': 'dead by daylight',
  'day': 20,
  'month': 6,
  'year': 2019},
 {'max_retweets': 3442,
  'game': 'dead by daylight',
  'day': 21,
  'month': 6,
  'year': 2019},
 {'max_retweets': 1303,
  'game': 'minecraft',
  'day': 22,
  'month': 6,
  'year': 2019},
 {'max_retweets': 2149, 'game': 'slots', 'day': 23, 'month': 6, 'year': 2019},
 {'m

### Which is the game with the highest likes for each day?

In [72]:
pipeline_highest_likes_per_day = [{
		"$group": {
			"_id": {
				"name": "$query",
				"day": {
					"$dayOfMonth": "$date"
				},
				"month": {
					"$month": "$date"
				},
				"year": {
					"$year": "$date"
				}
			},
			"tot_likes": {
				"$sum": "$likes"
			}
		}
	},
	{
		"$sort": {
			"tot_likes": -1
		}
	},
	{
		"$group": {
			"_id": {
				"day": "$_id.day",
				"month": "$_id.month",
				"year": "$_id.year"
			},
			"max_likes": {
				"$first": "$tot_likes"
			},
			"game": {
				"$first": "$_id.name"
			}
		}
	},
	{
		"$project": {
			"_id": 0,
			"day": "$_id.day",
			"month": "$_id.month",
			"year": "$_id.year",
			"max_likes": 1,
			"game": 1

		}
	},
	{
		"$sort": {
			"day": 1
		}
	}
]
highest_likes_per_day = list(db.twitter.aggregate(pipeline_highest_likes_per_day))
highest_likes_per_day

[{'max_likes': 7344, 'game': 'fortnite', 'day': 13, 'month': 6, 'year': 2019},
 {'max_likes': 34285, 'game': 'fortnite', 'day': 14, 'month': 6, 'year': 2019},
 {'max_likes': 48678,
  'game': 'pokémon gold/silver',
  'day': 15,
  'month': 6,
  'year': 2019},
 {'max_likes': 33085, 'game': 'fortnite', 'day': 16, 'month': 6, 'year': 2019},
 {'max_likes': 35726, 'game': 'fortnite', 'day': 17, 'month': 6, 'year': 2019},
 {'max_likes': 53061, 'game': 'fortnite', 'day': 18, 'month': 6, 'year': 2019},
 {'max_likes': 15202, 'game': 'fortnite', 'day': 19, 'month': 6, 'year': 2019},
 {'max_likes': 30358, 'game': 'fortnite', 'day': 20, 'month': 6, 'year': 2019},
 {'max_likes': 22664, 'game': 'fortnite', 'day': 21, 'month': 6, 'year': 2019},
 {'max_likes': 43675,
  'game': 'minecraft',
  'day': 22,
  'month': 6,
  'year': 2019},
 {'max_likes': 25375,
  'game': 'minecraft',
  'day': 23,
  'month': 6,
  'year': 2019},
 {'max_likes': 39009, 'game': 'fortnite', 'day': 24, 'month': 6, 'year': 2019}]

## Other queries that we may do

- How the top 25 games on VGChartz are performing on Twitter and Twitch (just for using multiple collections)
- Which is the day with the highest views/likes/retweets of all time? And which is the one with the lowest values?
- Which is the day of the week with the highest "buzz"? And which is the one with the lowest "buzz"?

In [ ]:
server.stop()